In [ ]:
### Trend and information flow of Russia propaganda
# https://www.kaggle.com/code/alifiyabatterywala/ukraine-vs-russian-topic-modeling/notebook


In [32]:
import pandas as pd 
import numpy as np
import os
import csv
import warnings
warnings.simplefilter(action='ignore')
import gc
from pathlib import Path
import math
import glob

In [2]:
# create a filelist for reading .gzip datasets 

# method 1
data_location = os.path.join('kaggle', '*.gzip')

filenames = glob.glob(data_location)
print(filenames)

['kaggle/UkraineCombinedTweetsDeduped_MAR05.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR04.csv.gzip', 'kaggle/0401_UkraineCombinedTweetsDeduped.csv.gzip', 'kaggle/0402_UkraineCombinedTweetsDeduped.csv.gzip', 'kaggle/0403_UkraineCombinedTweetsDeduped.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_FEB27.csv.gzip', 'kaggle/0405_UkraineCombinedTweetsDeduped.csv.gzip', 'kaggle/0404_UkraineCombinedTweetsDeduped.csv.gzip', 'kaggle/0407_UkraineCombinedTweetsDeduped.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR31.csv.gzip', 'kaggle/0406_UkraineCombinedTweetsDeduped.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR12.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR13.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR18.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR19.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR26.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR15.csv.gzip', 'kaggle/UkraineCombinedTweetsDeduped_MAR14.csv.gzip', 'kaggle/UkraineCombinedTweetsDedup

In [19]:
# search for tweets with keywords 

columns = ["userid","username","location","language","tweetid","tweetcreatedts","retweetcount", "hashtags", "text", "usercreatedts", "following", "followers","totaltweets"]

filenames.sort()
dataframe_collection = []
# create empty dataframes for subtopics
dfnames = ['df_smo', 'df_R_smo','df_unazi','df_nazi']
for x in dfnames: exec(x + ' = pd.DataFrame()')

for csvfile in filenames:
    df = pd.read_csv(csvfile, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
    df = df[columns]
    # key word selection
    df_smo = pd.concat([df_smo, df[df['text'].str.contains('special military operation')]])
    df_R_smo = pd.concat([df_R_smo, df[df['text'].str.contains('специальная военная операция')]])
    df_unazi = pd.concat([df_unazi, df[df['text'].str.contains('cleanse of Nazis'or 'cleanse of nazis'or 'Ukrainian Nazis'or 'Ukrainian nazis' or 'Ukrainian nazi' or 'Ukrainian Nazi')]])
    df_nazi = pd.concat([df_nazi, df[df['text'].str.contains('Neo-Nazis' or 'fascists')] ])
    
gc.collect()



634

In [20]:
# combine datasets from different days
print(len(df_smo),len(df_R_smo), len(df_unazi), len(df_nazi))

11918 4 0 5419


In [25]:
# a function to split and clean "hashtags" column

def SplitTags(hashtags):
    TAGS=[]
    if hashtags != '[]':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]
        return TAGS
    else: return "NA"

def LowerTags(hashtags):
    TAGS=[]
    if hashtags != '[]':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]
    lower_TAGS = (map(lambda x: x.lower(), TAGS))
    TAGS = list(lower_TAGS)
    if len(TAGS)>0:
        return TAGS
    else: return "NA"

            
df_smo['tags_splitted'] = df_smo['hashtags'].apply(SplitTags)
df_smo['tags_lower'] = df_smo['hashtags'].apply(LowerTags)

,userid,username,location,language,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets,tags_splitted,tags_lower
373,1123400778,WizardBear1,"Hawaii, USA",en,1509682342587109379,2022-04-01 00:01:33.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2013-01-26 23:10:16.000000,1289,1019,78199,NA,NA
907,1725818820,MaureenStaehle,"Nova Scotia, Canada",en,1509683012065275910,2022-04-01 00:04:13.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2013-09-03 15:32:02.000000,5005,1313,39329,NA,NA
914,28997676,BobDarrin,"Meridian, Mississippi",en,1509683024497156102,2022-04-01 00:04:16.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2009-04-05 14:47:55.000000,31125,34448,96475,NA,NA
943,1310329297966571520,Serafina_chi,NaN,en,1509683058445750285,2022-04-01 00:04:24.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2020-09-27 21:24:14.000000,12545,12490,5238,NA,NA
1057,1252704661385154560,jrzgrlnsunshine,"Florida, USA",en,1509683207544967180,2022-04-01 00:04:59.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2020-04-21 21:04:15.000000,1502,443,23932,NA,NA
1467,1432058723460071433,Michael92837265,Swfla,en,1509683743308623885,2022-04-01 00:07:07.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2021-08-29 19:13:17.000000,1258,1058,20921,NA,NA
1939,1691478300,kathies01,NaN,en,1509684387213885441,2022-04-01 00:09:41.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2013-08-22 16:28:39.000000,5157,5436,286469,NA,NA
2018,164160602,Marisol_Maddox,"Washington, DC",en,1509684510509699078,2022-04-01 00:10:10.000000,0,"[{'text': 'Russia', 'indices': [61, 68]}, {'te...",This includes facing up to 15 years in jail if...,2010-07-08 05:15:53.000000,2679,2854,12516,"[Russia, Ukraine]","[russia, ukraine]"
2602,1284749755193540608,Peninsula360P,"Redwood City, CA",en,1509685375886843906,2022-04-01 00:13:36.000000,0,"[{'text': 'Russian', 'indices': [70, 78]}, {'t...",Webcast tomorrow! April 1st at 10:00 am Pacifi...,2020-07-19 07:21:15.000000,395,230,2173,"[Russian, Ukraine, VladimirPutin]","[russian, ukraine, vladimirputin]"
2785,907139600,Isellmpls,"Minnesota, USA",en,1509685620574097408,2022-04-01 00:14:35.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2012-10-27 01:35:14.000000,30896,34491,138689,NA,NA


In [26]:
# import the manually labelled new hashtag list
with open("labelled.txt", 'r', encoding='UTF-8') as infile:
    lines = infile.readlines()

    labelled_tags = pd.DataFrame(columns=lines[0].split())
    
    for line in lines[1:]:
        df_length = len(labelled_tags)
        labelled_tags.loc[df_length] = line.split()[1:]
        
labelled_tags["pro_u"] = labelled_tags["pro_u"].apply(lambda x: eval(x))
labelled_tags["pro_r"] = labelled_tags["pro_r"].apply(lambda x: eval(x))
labelled_tags["neutral"] = labelled_tags["neutral"].apply(lambda x: eval(x))
labelled_tags.head()

pro_russian_tags = labelled_tags["tag"][labelled_tags["pro_r"]==True]
pro_ukrainian_tags = labelled_tags["tag"][labelled_tags["pro_u"]==True]
neural_tags = labelled_tags["tag"][labelled_tags["neutral"]==True]


In [27]:
# change all tags to lowercase before matching to avoid spelling difference
lower_R = (map(lambda x: x.lower(), pro_russian_tags))
pro_russian_tags = list(lower_R)

lower_U = (map(lambda x: x.lower(), pro_ukrainian_tags))
pro_ukrainian_tags = list(lower_U)

lower_N = (map(lambda x: x.lower(), neural_tags))
neural_tags = list(lower_N)

In [28]:
# Create lambda functions to mark political orientation if related hashtags were found 

# lambda functions marking political orientation if related hashtags were found 
pro_R_check = lambda taglist: bool(set(taglist) & set(pro_russian_tags)) # return true if overlap of two sets not empty
pro_U_check = lambda taglist: bool(set(taglist) & set(pro_ukrainian_tags)) 
N_check = lambda taglist: bool(set(taglist) & set(neural_tags)) 


df_smo['pro_Russian'] = df_smo['tags_lower'].apply(pro_R_check)
df_smo['pro_Ukraine'] = df_smo['tags_lower'].apply(pro_U_check)
df_smo['Neutral'] = df_smo['tags_lower'].apply(N_check)

In [33]:
# Determine a tweet's stance by it's hashtags: pro-russian / pro-ukrainian / both/ Neutral/ unknown
df_smo['NodeType'] = np.where(
     df_smo['pro_Russian'] & df_smo['pro_Ukraine'], 'Both',
     np.where(
         df_smo['pro_Russian'],  'pro_Russian',
     np.where(
        df_smo['pro_Ukraine'], 'pro_Ukraine', 
     np.where(
         df_smo['Neutral'],'Neutral','Unknown'
     ))))

df_smo.head(2)

,userid,username,location,language,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets,tags_splitted,tags_lower,pro_Russian,pro_Ukraine,Neutral,NodeType
373,1123400778,WizardBear1,"Hawaii, USA",en,1509682342587109379,2022-04-01 00:01:33.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2013-01-26 23:10:16.000000,1289,1019,78199,NA,NA,False,False,False,Unknown
907,1725818820,MaureenStaehle,"Nova Scotia, Canada",en,1509683012065275910,2022-04-01 00:04:13.000000,151,[],The sounds of war in Ukraine. \n\nPutin calls ...,2013-09-03 15:32:02.000000,5005,1313,39329,NA,NA,False,False,False,Unknown


In [36]:
#plot time distribution of tweets
import matplotlib.pyplot as plt
import seaborn as sns

# Distribution of tweet created time by political stance
fig, ax = plt.subplots(figsize=(16,8))
for stance in df_smo['NodeType'].value_counts().sort_values().index.tolist():
    sns.kdeplot(df_smo[df_smo['NodeType']==stance]['tweetcreatedts'],ax=ax, label=stance)
ax.legend()
ax.set_title("Distribution of tweet created time by political stance")
plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [39]:
# export dataframe 
df1.to_csv("20220502_resampled_dataset.csv",sep = ",", header = True, encoding = "UTF-8",index=False)